In [1]:
from pytorch_model_summary import summary

In [2]:
import torch

In [8]:
from model import import_model
from argparse import Namespace
opt = Namespace()

opt.config = {
    'model': {
        'name': 'retinex',  # or 'mobileie', 'retinex'
        'type': 'original',  # or 're-parameterized'
        'channels': 12,
        'rep_scale': 4,
        'pretrained': None,
        'pretrained_date': None,
        'need_slim': False,
    }
}
opt.model_task = 'isp'
opt.device = 'cpu'

retinex_model = import_model(opt)


In [9]:
print(summary(retinex_model, torch.zeros((1, 4, 512, 512)), show_input=False))

--------------------------------------------------------------------------
      Layer (type)           Output Shape         Param #     Tr. Param #
        MBRConv5-1      [1, 12, 512, 512]          20,124          14,364
        MBRConv3-2      [1, 12, 512, 512]          19,020          14,412
      RetinexFST-3      [1, 12, 512, 512]          44,112          32,592
     RetinexHDPA-4      [1, 12, 512, 512]           7,382           4,502
        MBRConv3-5      [1, 12, 512, 512]          19,020          14,412
      RetinexFST-6      [1, 12, 512, 512]          44,112          32,592
     RetinexHDPA-7      [1, 12, 512, 512]           7,382           4,502
        MBRConv1-8      [1, 12, 512, 512]           3,036           1,884
    PixelShuffle-9     [1, 3, 1024, 1024]               0               0
       MBRConv1-10     [1, 3, 1024, 1024]             219             147
        Sigmoid-11     [1, 3, 1024, 1024]               0               0
Total params: 164,407
Trainable param

In [5]:
slim_model = retinex_model.slim()

In [6]:
print(summary(slim_model, torch.zeros((1, 4, 512, 512)), show_input=False))

--------------------------------------------------------------------------
      Layer (type)           Output Shape         Param #     Tr. Param #
          Conv2d-1      [1, 12, 512, 512]           1,212           1,212
     RetinexFSTS-2      [1, 12, 512, 512]           1,476           1,476
          Conv2d-3      [1, 12, 512, 512]           1,308           1,308
     RetinexFSTS-4      [1, 12, 512, 512]           1,476           1,476
    RetinexHDPAS-5      [1, 12, 512, 512]             312             312
          Conv2d-6      [1, 12, 512, 512]           1,308           1,308
     RetinexFSTS-7      [1, 12, 512, 512]           1,476           1,476
    RetinexHDPAS-8      [1, 12, 512, 512]             312             312
          Conv2d-9      [1, 12, 512, 512]             156             156
   PixelShuffle-10     [1, 3, 1024, 1024]               0               0
         Conv2d-11     [1, 3, 1024, 1024]              12              12
        Sigmoid-12     [1, 3, 1024, 1

In [7]:
retinex_model.eval()
slim_model.eval();

In [11]:
input = torch.randn(1, 4, 448, 448)

# Write a time checking by inferences 20 times
import time 

start_time = time.time()
for _ in range(20):
    output1 = slim_model(input)
end_time = time.time()
print(f"Retinex model average inference time: {(end_time - start_time) / 20:.6f} seconds")

Retinex model average inference time: 0.117111 seconds
